In [ ]:
!pip install -U kaleido

In [8]:
import os
import pandas as pd
import plotly.express as px

In [22]:
df = pd.read_csv("./dataset_final.csv", index_col=0)
df

,order_purchase_timestamp,time_to_delivery_from_shipment,time_to_delivery_from_order_approved,time_to_delivery_from_order_purchased,freight_value,shipment_distance
0,2016-09-15 12:16:38,1.608160,54.813194,54.813194,2.83,564.407287
1,2016-10-03 09:44:50,3.000000,19.924525,23.178738,15.56,709.921605
2,2016-10-03 16:56:50,6.073519,21.094375,24.057500,17.19,914.131568
3,2016-10-03 21:13:36,8.958426,29.323819,30.572581,17.24,816.498486
4,2016-10-03 22:06:03,9.863947,27.027488,27.542813,13.56,212.084845
...,...,...,...,...,...,...
92817,2018-08-29 12:25:59,1.382257,1.425810,1.432269,9.20,18.854775
92818,2018-08-29 14:18:23,1.038831,1.079028,1.087870,46.48,10.343719
92819,2018-08-29 14:18:28,0.917720,1.098704,1.106979,15.39,431.976676
92820,2018-08-29 14:52:00,0.858322,1.063623,1.072905,8.33,10.312870


In [23]:
ts_columns = [
    "time_to_delivery_from_order_approved", 
    "time_to_delivery_from_shipment", 
    "time_to_delivery_from_order_purchased",
    "freight_value",
    "shipment_distance"
]

def plot_ts_features(df, ts_columns, prefix):
    if not os.path.exists("images"):
        os.mkdir("images")
    for col in ts_columns:
        fig = px.line(df, x='order_purchase_timestamp', y=col)
        fig.show()
        fig.write_html(f"./images/{prefix}_{col}.html",
                full_html=False,
                include_plotlyjs='cdn')

In [24]:
df = df[df["order_purchase_timestamp"] > '2017-01-05 00:00:00']
df

,order_purchase_timestamp,time_to_delivery_from_shipment,time_to_delivery_from_order_approved,time_to_delivery_from_order_purchased,freight_value,shipment_distance
258,2017-01-05 11:56:06,5.021111,6.044306,6.054155,8.72,11.609337
259,2017-01-05 12:01:20,5.969502,10.502326,12.151134,8.72,11.609337
260,2017-01-05 12:06:36,5.020463,9.513889,11.165810,8.72,13.076936
261,2017-01-05 12:09:08,5.005278,9.499549,11.148854,8.72,13.076936
262,2017-01-05 12:11:23,5.988623,10.515544,12.163275,8.72,13.076936
...,...,...,...,...,...,...
92817,2018-08-29 12:25:59,1.382257,1.425810,1.432269,9.20,18.854775
92818,2018-08-29 14:18:23,1.038831,1.079028,1.087870,46.48,10.343719
92819,2018-08-29 14:18:28,0.917720,1.098704,1.106979,15.39,431.976676
92820,2018-08-29 14:52:00,0.858322,1.063623,1.072905,8.33,10.312870


In [ ]:
plot_ts_features(df, ts_columns, "date_filtered")

In [46]:
import numpy as np
def remove_outliers_by_iqr(df, column, whisker_width=1.5):
    df_outliers = df.copy()
    q1 = df_outliers[column].quantile(0.25)              
    q3 = df_outliers[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - whisker_width * iqr
    upper_bound = q3 + whisker_width * iqr
    p50 = df_outliers[column].median()
    df_outliers[col] = df_outliers[col].apply(lambda x: p50 if x <= lower_bound or x >= upper_bound else x)
    #filter = (df_outliers[column] >= lower_bound) & (df_outliers[column] <= upper_bound)
    #df_outliers[~filter] = np.nan
    df_outliers.dropna()
    df_outliers[column].fillna(p50, inplace=True)
    return df_outliers

df_without_outliers = df.copy()
for col in ts_columns:
    df_without_outliers = remove_outliers_by_iqr(df_without_outliers, col)
df_without_outliers = df_without_outliers.reset_index(drop=True)
df_without_outliers

,order_purchase_timestamp,time_to_delivery_from_shipment,time_to_delivery_from_order_approved,time_to_delivery_from_order_purchased,freight_value,shipment_distance
0,2017-01-05 11:56:06,5.021111,6.044306,6.054155,8.72,11.609337
1,2017-01-05 12:01:20,5.969502,10.502326,12.151134,8.72,11.609337
2,2017-01-05 12:06:36,5.020463,9.513889,11.165810,8.72,13.076936
3,2017-01-05 12:09:08,5.005278,9.499549,11.148854,8.72,13.076936
4,2017-01-05 12:11:23,5.988623,10.515544,12.163275,8.72,13.076936
...,...,...,...,...,...,...
92470,2018-08-29 12:25:59,1.382257,1.425810,1.432269,9.20,18.854775
92471,2018-08-29 14:18:23,1.038831,1.079028,1.087870,16.43,10.343719
92472,2018-08-29 14:18:28,0.917720,1.098704,1.106979,15.39,431.976676
92473,2018-08-29 14:52:00,0.858322,1.063623,1.072905,8.33,10.312870


In [47]:
df_without_outliers.describe()

,time_to_delivery_from_shipment,time_to_delivery_from_order_approved,time_to_delivery_from_order_purchased,freight_value,shipment_distance
count,92475.000000,92475.000000,92475.000000,92475.000000,92475.000000
mean,7.893395,10.580126,11.035263,16.300623,463.908649
std,5.121525,5.845805,5.945328,5.031114,359.553576
min,0.000012,0.007662,0.533414,1.980000,0.000000
25%,4.127297,6.328791,6.784890,13.410000,184.756918
50%,7.118449,9.879062,10.244155,16.430000,433.987729
75%,10.740035,13.818628,14.235029,18.370000,642.847273
max,23.971887,28.421736,29.209653,33.180000,1721.954595


In [ ]:
plot_ts_features(df_without_outliers, ts_columns, "without_outliers")

In [15]:
!pip install statsmodels

                                              0.0/9.2 MB ? eta -:--:--
                                              0.0/9.2 MB ? eta -:--:--
                                              0.0/9.2 MB 435.7 kB/s eta 0:00:22
                                              0.1/9.2 MB 744.7 kB/s eta 0:00:13
                                              0.2/9.2 MB 1.2 MB/s eta 0:00:08
     -                                        0.3/9.2 MB 1.5 MB/s eta 0:00:06
     --                                       0.6/9.2 MB 2.2 MB/s eta 0:00:04
     ----                                     1.0/9.2 MB 3.4 MB/s eta 0:00:03
     -----                                    1.4/9.2 MB 3.9 MB/s eta 0:00:02
     -------                                  1.7/9.2 MB 4.4 MB/s eta 0:00:02
     ---------                                2.2/9.2 MB 5.0 MB/s eta 0:00:02
     -----------                              2.7/9.2 MB 5.6 MB/s eta 0:00:02
     -------------                            3.2/9.2 MB 5.9 MB/s eta 0:00

In [16]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [59]:
df_without_outliers = df_without_outliers.set_index('order_purchase_timestamp')
df_without_outliers.index = pd.to_datetime(df_without_outliers.index)

In [60]:
df_without_outliers

,time_to_delivery_from_shipment,time_to_delivery_from_order_approved,time_to_delivery_from_order_purchased,freight_value,shipment_distance
order_purchase_timestamp,,,,,
2017-01-05 11:56:06,5.021111,6.044306,6.054155,8.72,11.609337
2017-01-05 12:01:20,5.969502,10.502326,12.151134,8.72,11.609337
2017-01-05 12:06:36,5.020463,9.513889,11.165810,8.72,13.076936
2017-01-05 12:09:08,5.005278,9.499549,11.148854,8.72,13.076936
2017-01-05 12:11:23,5.988623,10.515544,12.163275,8.72,13.076936
...,...,...,...,...,...
2018-08-29 12:25:59,1.382257,1.425810,1.432269,9.20,18.854775
2018-08-29 14:18:23,1.038831,1.079028,1.087870,16.43,10.343719
2018-08-29 14:18:28,0.917720,1.098704,1.106979,15.39,431.976676


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import DecomposeResult, seasonal_decompose


def plot_seasonal_decompose(result: DecomposeResult, title="Seasonal Decomposition"):
    return (
        make_subplots(
            rows=4,
            cols=1,
            subplot_titles=["Observed", "Trend", "Seasonal", "Residuals"],
        )
        .add_trace(
            go.Scatter(x=result.seasonal.index, y=result.observed, mode="lines"),
            row=1,
            col=1,
        )
        .add_trace(
            go.Scatter(x=result.trend.index, y=result.trend, mode="lines"),
            row=2,
            col=1,
        )
        .add_trace(
            go.Scatter(x=result.seasonal.index, y=result.seasonal, mode="lines"),
            row=3,
            col=1,
        )
        .add_trace(
            go.Scatter(x=result.resid.index, y=result.resid, mode="lines"),
            row=4,
            col=1,
        )
        .update_layout(
            height=900, title=title, margin=dict(t=100), title_x=0.5, showlegend=False
        )
    )

for model_type in ["additive", "multiplicative"]:
    for col in ts_columns:
        fig = plot_seasonal_decompose(
            seasonal_decompose(df_without_outliers[col], model=model_type, period=12),
            f"Seasonal Decomposition - {model_type} - {col}"
        )
        fig.show()
        fig.write_html(f"./images/decomposition_{model_type}_{col}.html",
                        full_html=False,
                        include_plotlyjs='cdn')

In [69]:
df_without_outliers.to_csv("./dataset_pre_processed.csv")